In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from functools import partial

# Plot training and validation metrics
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(121)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [4]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
import time

In [11]:
start_time = time.time()
# Set your data directory
data_directory = '/Users/shreyas/Desktop/5th_sem/DeepLearning_Lab/Project/training_experimental/data/train'  # Update this path to the directory containing your dataset

# Define constants
input_shape = (48, 48, 1)  # Updated input size to match your image size
num_classes = 7  # Automatically determine the number of classes
batch_size = 8192
epochs = 20

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to the [0, 1] range
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and preprocess the data
datagen_with_preprocessing = datagen.flow_from_directory(
    data_directory,
    target_size=(48, 48),  # Updated target size
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

# Split the data into training and validation sets
X_train, y_train = datagen_with_preprocessing.next()
X_val, y_val = datagen_with_preprocessing.next()

def create_model(optimizer, loss):
    model = keras.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=optimizer, loss=loss, metrics=['categorical_accuracy'])

    return model

model = KerasClassifier(model=create_model, verbose=1)

# Define hyperparameter options
param_grid = {
    'optimizer': ['adam', 'rmsprop', 'sgd', 'adagrad'],
    'loss': ['categorical_crossentropy', 'kullback_leibler_divergence', 'squared_hinge', 'huber_loss'],
    'batch_size': [8192, 16384, 32768],
    'epochs': [20, 50, 100]
}

best_accuracy = 0
best_params = {}

for params in ParameterGrid(param_grid):
    optimizer = params['optimizer']
    loss = params['loss']
    batch_size = params['batch_size']
    epochs = params['epochs']

    # Create and compile the model with current hyperparameters
    model = create_model(optimizer, loss)
    
    print("Optimizer: ", optimizer, "\nLoss: ", loss, "\nBatch Size: ", batch_size, "\nEpochs: ", epochs)

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, verbose=1)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_val, y_val)

    # Check if this set of hyperparameters resulted in a better accuracy
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_params = params

print("Best Parameters:", best_params)
print("Best Accuracy: {:.2f}%".format(best_accuracy * 100))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total training time: {elapsed_time:.2f} seconds")

Found 28709 images belonging to 7 classes.
Optimizer:  adam 
Loss:  categorical_crossentropy 
Batch Size:  8192 
Epochs:  20
Epoch 1/20
1/1 [==============================] - 5s 5s/step - loss: 1.9463 - categorical_accuracy: 0.1469 - val_loss: 1.8384 - val_categorical_accuracy: 0.2502
Epoch 2/20
1/1 [==============================] - 4s 4s/step - loss: 1.8359 - categorical_accuracy: 0.2531 - val_loss: 1.8287 - val_categorical_accuracy: 0.2502
Epoch 3/20
1/1 [==============================] - 4s 4s/step - loss: 1.8217 - categorical_accuracy: 0.2532 - val_loss: 1.8309 - val_categorical_accuracy: 0.2533
Epoch 4/20
1/1 [==============================] - 3s 3s/step - loss: 1.8227 - categorical_accuracy: 0.2499 - val_loss: 1.8317 - val_categorical_accuracy: 0.2502
Epoch 5/20
1/1 [==============================] - 3s 3s/step - loss: 1.8231 - categorical_accuracy: 0.2529 - val_loss: 1.8304 - val_categorical_accuracy: 0.2502
Epoch 6/20
1/1 [==============================] - 3s 3s/step - loss: 1